## Seq2Seq Learning with Neural Networks (NIPS 2014) 실습

### 1. 데이터 전처리
  - spaCy library: 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

In [50]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [51]:
import spacy

# 영어 및 독일어 토큰화
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

In [52]:
# 간단한 토큰화 수행
tokenized = spacy_en.tokenizer('I am a graduate student.')

for idx, token in enumerate(tokenized):
  print(f'index {idx}: {token.text}')

index 0: I
index 1: am
index 2: a
index 3: graduate
index 4: student
index 5: .


In [53]:
# 영어 및 독일어 토큰화 함수 정의

# 독일어 문장 (입력 시퀀스) 토큰화 한 뒤, 순서 뒤집기
# 논문에서 인코더를 거꾸로 뒤집을 때 성능이 좋게 나왔다고 함
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어 문장 (출력 시퀀스) 토큰화 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [54]:
!pip install -U torchtext==0.8.0

In [55]:
# 필드 라이브러리를 이용한 데이터셋의 구체적인 전처리 내용 명시
from torchtext.data import Field, BucketIterator

# 소스 : 독일어
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)

# 목표 : 영어
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [56]:
# 영어-독어 번역 데이터셋 임포트
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [57]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [58]:
# 학습 데이터 중 하나를 선택해 출력
# 훈련 데이터의 src 필드 단어 예시 => 독어 (순서 뒤집기)
print(vars(train_dataset.examples[30])['src'])

# 훈련 데이터의 trg 필드 단어 예시 => 영어
print(vars(train_dataset.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [60]:
# field 객체의 build_vocab 메소드를 이용하여 영어, 독어 단어 사전 생성
# 최소 2번 이상 등장한 단어만 선택

SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f'len(SRC): {len(SRC.vocab)}')
print(f'len(TRG): {len(TRG.vocab)}')

len(SRC): 7855
len(TRG): 5893


In [62]:
# 생성된 단어 집합 안에 있는 단어들 확인
print(TRG.vocab.stoi['abcabc']) # 없는 단어 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding) : 1
print(TRG.vocab.stoi['<sos>']) # <sos>: 2
print(TRG.vocab.stoi['<eos>']) # <eos>: 3
print(TRG.vocab.stoi['hello'])
print(TRG.vocab.stoi['world'])

0
1
2
3
4112
1752


In [65]:
'''
  - 한 문장에 포함된 단어가 연속적으로 LSTM에 입력되어야 함
    1. 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋음
    2. 이를 위하여 BucketIterator 사용
    3. 배치 크기: 128
'''

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 데이터 로더의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [76]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f'first batch size: {src.shape}')

  # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
  for i in range(src.shape[0]):
    print(f'index {i}: {src[i][0].item()}')

  # check only first batch
  break

first batch size: torch.Size([26, 128])
index 0: 2
index 1: 4
index 2: 317
index 3: 24
index 4: 12
index 5: 1769
index 6: 168
index 7: 8
index 8: 93
index 9: 10
index 10: 128
index 11: 558
index 12: 5
index 13: 61
index 14: 16
index 15: 955
index 16: 8
index 17: 3
index 18: 1
index 19: 1
index 20: 1
index 21: 1
index 22: 1
index 23: 1
index 24: 1
index 25: 1


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


### 2. 인코더 아키텍처
  - 주어진 소스 문장을 context vector로 인코딩
  - LSTM은 hidden state와 cell state를 반환
  - hyper parameters
    - input_dim: 하나의 단어에 대한 원핫 인코딩 차원
    - embed_dim: 임베딩 차원
    - hidden_dim: 히든 상태 차원
    - n_layers: RNN 레이어의 개수
    - dropout_ratio: 드롭아웃 비율

In [ ]:
import torch.nn as nn

# 인코더 아키텍처 정의
class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio) -> None:
      super().__init__()

      # 임베딩은 원-핫 인코딩을 특정 차원의 임베딩으로 매핑하는 레이어
      self.embedding = nn.Embedding(input_dim, embed_dim)

      # LSTM layer
      self.hidden_dim = hidden_dim
      self.n_layers = n_layers
      self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

      # dropout
      self.dropout = nn.Dropout(dropout_ratio)

  # 인코더는 소스 문장을 입력으로 받아 문맥 벡터를 반환
  def forward(self, src):
    # src: [단어 개수, 배치 크기]: 각 단어의 인덱스 정보
    # embedded: [단어 개수, 배치 크기, 임베딩 차원]
    embedded = self.dropout(self.embedding(src))

    # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
    # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
    # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
    outputs, (hidden, cell) = self.rnn(embedded)

    # return context vector
    return hidden, cell

### 3. 디코더 아키텍처
  - 주어진 문맥 벡터를 타겟 문장으로 디코딩
  - LSTM은 hidden state와 cell state를 반환
  - hyper parameters
    - input_dim: 하나의 단어에 대한 원핫 인코딩 차원
    - embed_dim: 임베딩(embedding) 차원
    - hidden_dim: 히든 상태(hidden state) 차원
    - n_layers: RNN 레이어의 개수
    - dropout_ratio: 드롭아웃(dropout) 비율

In [ ]:
# 디코더 아키텍처 정의
class Decoder(nn.Module):
  def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio) -> None:
      super().__init__()

      # 임베딩은 원-핫 인코딩 말고 특정 차원의 임베딩으로 매핑하는 레이어
      self.embedding = nn.Embedding(output_dim, embed_dim)

      # LSTM layer
      self.hidden_dim = hidden_dim
      self.n_layers = n_layers
      self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

      # FC layer (인코더와 구조적으로 다른 부분)
      self.output_dim = output_dim
      self.fc_out = nn.Linear(hidden_dim, output_dim)

      # dropout
      self.dropout = nn.Dropout(dropout_ratio)

  # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
  def forward(self, input, hidden, cell):
      # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
      # hidden: [레이어 개수, 배치 크기, 히든 차원]
      # cell = context: [레이어 개수, 배치 크기, 히든 차원]
      input = input.unsqueeze(0)
      # input: [단어 개수 = 1, 배치 크기]

      # embedded: [단어 개수, 배치 크기, 임베딩 차원]
      embedded = self.dropout(self.embedding(input))

      # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
      # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
      # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
      output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

      # 단어 개수는 1개이므로 차원 제거
      # prediction = [배치 크기, 출력 차원]
      prediction = self.fc_out(output.squeeze(0))

      # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
      return prediction, hidden, cell

### 4. Seq2Seq 아키텍처
  - encoder: 주어진 소스 문장을 문맥 벡터로 인코딩
  - decoder: 주어진 문맥 벡터를 타겟 문장으로 디코딩
  - 디코더는 한 단어씩 넣어서 한 번씩 결과를 구함
  - teacher forcing: 디코더의 예측을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용하는 기법

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device) -> None:
      super().__init__()

      self.encoder = encoder
      self.decoder = decoder
      self.device = device

  # 학습 시, 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ration 넣어야 함
  def forward(self, src, trg, teacher_forcing_ratio=0.5):
    # src: [단어 개수, 배치 크기]
    # trg: [단어 개수, 배치 크기]
    # 먼저 인코더를 거쳐 문맥 벡터를 추출
    hidden, cell = self.encoder(src)

    # decoder의 최종 결과를 담을 텐서 객체 생성
    trg_len = trg.shape[0] # 단어 개수
    batch_size = trg.shape[1] # 배치 크기
    trg_vocab_size = self.decoder.output_dim # 출력 차원
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    # 첫 번째 입력은 항상 <sos> 토큰
    input = trg[0, :]

    # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
    for t in range(1, trg_len):
      output, hidden, cell = self.decoder(input, hidden, cell)

      outputs[t] = output # FC를 거쳐 나온 현재 출력 단어 정보
      top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

      # teacher_forcing_ratio: 학습 시 실제 목표 출력(ground-truth)을 사용하는 비율
      import random
      teacher_force = random.random() < teacher_forcing_ratio
      input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에 넣기

    return outputs

### 5. 학습
  - 하이퍼 파라미터 설정 및 모델 초기화

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [ ]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
from imp import init_builtin
# 본 논문의 내용대로 u(-0.08, 0.08)의 값으로 모델 가중치 파라미터 초기화
def init_weights(m):
  for name, param in m.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
# 학습 및 평가 함수 정의
import torch.optim as optim

# adam optimizer 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
# 모델 학습 함수
def train(model, iterator, optimizer, criterion, clip):
  model.train() # 학습 모드
  epoch_loss = 0

  # 전체 학습 데이터 확인
  for i, batch in enumerate(iterator):
    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()

    # output: [출력 단어 개수, 배치 크기, 출력 차원]
    output = model(src, trg)
    output_dim = output.shape[-1]

    # 출력 단어의 인덱스 0은 사용하지 않음
    # output = [(출력 단어의 개수 - 1) * batch size, output dim]
    output = output[1:].view(-1, output_dim)
    # trg = [(타겟 단어의 개수 - 1) * batch size]
    trg = trg[1:].view(-1)

    # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
    loss = criterion(output, trg)
    loss.backward() # 기울기 계산

    # 기울기 clipping 진행
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    # 파라미터 업데이트
    optimizer.step()

    # 전체 손실 값 계산
    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
# 모델 평가 함수
def evaluate(model, iterator, criterion):
  model.eval() # 평가 모드
  epoch_loss = 0

  with torch.no_grad():
    # 전체 평가 데이터를 확인하며
    for i, batch in enumerate(iterator):
      src = batch.src
      trg = batch.trg

      # 평가할 때 teacher forcing은 사용하지 않음
      # output: [출력 단어 개수, 배치 크기, 출력 차원]
      output = model(src, trg, 0)
      output_dim = output.shape[-1]

      # 출력 단어의 인덱스 0은 사용하지 않음
      # output = [(출력 단어의 개수 - 1) * batch size, output dim]
      output = output[1:].view(-1, output_dim)

      # trg = [(타겟 단어의 개수 - 1) * batch size]
      trg = trg[1:].view(-1)

      # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
      loss = criterion(output, trg)

      # 전체 손실 값 계산
      epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
# 학습 및 검증 시행
# epoch=20
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  # 시작 시간
  start_time = time.time()

  train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
  valid_loss = evaluate(model, valid_iterator, criterion)

  # 종료 시간
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'seq2seq.pt')

  print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
  print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Time: 0m 26s
	Train Loss: 5.065 | Train PPL: 158.352
	Validation Loss: 4.924 | Validation PPL: 137.519
Epoch: 02 | Time: 0m 26s
	Train Loss: 4.508 | Train PPL: 90.767
	Validation Loss: 4.888 | Validation PPL: 132.748
Epoch: 03 | Time: 0m 26s
	Train Loss: 4.170 | Train PPL: 64.738
	Validation Loss: 4.609 | Validation PPL: 100.377
Epoch: 04 | Time: 0m 26s
	Train Loss: 3.961 | Train PPL: 52.491
	Validation Loss: 4.569 | Validation PPL: 96.430
Epoch: 05 | Time: 0m 26s
	Train Loss: 3.784 | Train PPL: 43.998
	Validation Loss: 4.366 | Validation PPL: 78.709
Epoch: 06 | Time: 0m 27s
	Train Loss: 3.639 | Train PPL: 38.036
	Validation Loss: 4.252 | Validation PPL: 70.254
Epoch: 07 | Time: 0m 26s
	Train Loss: 3.497 | Train PPL: 33.023
	Validation Loss: 4.084 | Validation PPL: 59.358
Epoch: 08 | Time: 0m 27s
	Train Loss: 3.346 | Train PPL: 28.399
	Validation Loss: 3.991 | Validation PPL: 54.104
Epoch: 09 | Time: 0m 26s
	Train Loss: 3.216 | Train PPL: 24.926
	Validation Loss: 3.897 | Va

In [ ]:
# 학습된 모델 저장
from google.colab import files

files.download('seq2seq.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 6. 모델 최종 테스트 결과 확인

In [ ]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ERgwTMYWR7FMhApROaNvZREBTjEDi00ttSzt8ZNj1PS_5g?download=1 -O seq2seq.pt

--2022-03-25 06:26:09--  https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ERgwTMYWR7FMhApROaNvZREBTjEDi00ttSzt8ZNj1PS_5g?download=1
Resolving postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/dongbinna_postech_ac_kr/Documents/Research/models/seq2seq.pt [following]
--2022-03-25 06:26:10--  https://postechackr-my.sharepoint.com/personal/dongbinna_postech_ac_kr/Documents/Research/models/seq2seq.pt
Reusing existing connection to postechackr-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 55600205 (53M) [application/octet-stream]
Saving to: ‘seq2seq.pt’

seq2seq.pt          100%[===================>]  53.02M  14.5MB/s    in 3.6s    

2022-03-25 06:26:14 (14.5 MB/s) - ‘seq2seq.pt’ saved [55600205/55600205]



In [ ]:
import math
model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 3.593 | Test PPL: 36.330


In [ ]:
# translation function
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
  model.eval() # 평가 모드

  if isinstance(sentence, str):
    nlp = spacy.load('de')
    tokens = [token.text.lower() for token in nlp(sentence)]
  else:
    tokens = [token.lower() for token in sentence]

  # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
  tokens = [src_field.init_token] + tokens + [src_field.eos_token]
  print(f'전체 소스 토큰: {tokens}')

  src_indexes = [src_field.vocab.stoi[token] for token in tokens]
  print(f'소스 문장 인덱스: {src_indexes}')

  src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

  # 인코더에 소스 문장 넣어 문맥 벡터 계산
  with torch.no_grad():
    hidden, cell = model.encoder(src_tensor)

  # 처음에는 <sos> 토큰 하나만 가지고 있도록 함
  trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

  for i in range(max_len):
    # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

    with torch.no_grad():
      output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

    pred_token = output.argmax(1).item()
    trg_indexes.append(pred_token) # 출력 문장에 더하기

    # <eos> 만나는 순간 끝
    if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
      break

  # 각 출력 단어 인덱스를 실제 단어로 변환
  trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

  # 첫 번째 <sos>는 제외한 출력 문장 반환
  return trg_tokens[1:]

In [ ]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print('모델 출력 결과: ', ' '.join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', '.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine', '<eos>']
소스 문장 인덱스: [2, 4, 88, 20, 200, 780, 19, 565, 624, 70, 134, 10, 364, 8, 3]
모델 출력 결과:  a mother and her little boys enjoying a day day day . <eos>


In [ ]:
src = tokenize_de('Guten nacht')

print(f'소스 문장: {src}')
print('모델 출력 결과: ', ' '.join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['nacht', 'Guten']
전체 소스 토큰: ['<sos>', 'nacht', 'guten', '<eos>']
소스 문장 인덱스: [2, 521, 3799, 3]
모델 출력 결과:  at night . <eos>
